# Growth rate estimates

Code to read in information per country on total number of cases and number of days since case 100
Then to fit the data, obtaining the date of inflection and the initial growth rate

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import csv
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit

import pandas 

from scipy.optimize import curve_fit

%matplotlib inline

In [7]:
#Fit functions
def logistic_model(x,a,b,c):
    return c/(1+np.exp(-(x-b)/a))

def exponential_model(x,a,b,c):
    return a*np.exp(b*(x-c))

def linear_fit(x,m,c):
    return x*m + c

In [27]:
#Calculate chi-squared
def chisquare(data,expct):
    return sum((data-expct)**2 / expct)

In [43]:
#Function to obtain growth rate from data file
def ObtainGrowthRate(inputfile):
    
    #Read in data file 
    df = pandas.read_csv(inputfile)
    
    #Perform a linear fit to the data
    popt, pcov = curve_fit(polfit, df["Days_since_100"],np.log10(df["Total_cases"]))
    chi2 = chisquare(df["Total_cases"],10.**(df["Days_since_100"]*popt[0] + popt[1]))

    #Perform a logistic fit to the data
    log_fit, log_cov = curve_fit(logistic_model,df["Days_since_100"],df["Total_cases"],p0=[2.5,1,max(df["Total_cases"])])
    logchi2 = chisquare(df["Total_cases"],logistic_model(df["Days_since_100"],*log_fit))

    inflection_date = log_fit[1]

    if chi2 < logchi2 or inflection_date > max(df["Days_since_100"]):
        print("linear fit preferred")
        growthrate = popt[0]
        return growthrate

    print("logistic fit preferred")
    print("inflection date",inflection_date)
    
    #Perform linear fits pre and post inflection date to compare growth rates
    mask1 = df["Days_since_100"] < inflection_date
    mask2 = df["Days_since_100"] > inflection_date
    
    popt, pcov = curve_fit(polfit, df["Days_since_100"][mask1],np.log10(df["Total_cases"][mask1]))
    chi2 = chisquare(df["Total_cases"][mask1],10.**(df["Days_since_100"][mask1]*popt[0] + popt[1]))
    
    popt2, pcov2 = curve_fit(polfit, df["Days_since_100"][mask2],np.log10(df["Total_cases"][mask2]))
    chi2_2 = chisquare(df["Total_cases"][mask2],10.**(df["Days_since_100"][mask2]*popt2[0] + popt2[1]))


    growthrate = popt[0]
    growthrate2 = popt2[0]
    return growthrate, growthrate2
    

In [48]:
ObtainGrowthRate("Cases_China.csv")

logistic fit preferred
inflection date 21.30804312546522


(0.11434295276366868, 0.003796025616287113)